In [5]:
import numpy as np
import matplotlib.pyplot as plt
import cma
from es import SimpleGA, CMAES, PEPG, OpenES


def sigmoid(x):
    z = 1/(1 + np.exp(-x)) 
    return z

def BCE_loss(y,p):
    return np.sum(-y*np.log(p) - (1 - y)*np.log(1 - p))


class gate_layer():
    def __init__(self,dims, n_channels= 1, AND_OR = 1):
        
        # AND_OR = TRUE  : it is an AND gate
        # AND_OR = FALSE : it is an OR gate
        
        self.AND_OR = AND_OR
        self.n_channels = n_channels
        dims[0] = n_channels
        self.w =  np.random.normal(-3, 1, dims)
  
    def forward(self, X):
       
        X = np.repeat(X, self.n_channels, axis=0)

        m = sigmoid(self.w)

        if self.AND_OR == 1:
            filtered_X = 1 - m *(1 - X) 
            
            return np.prod(filtered_X,1)[np.newaxis,...]
        
        if self.AND_OR == 2:

            filtered_X = m*X
           
            return (1- np.prod(1 - filtered_X,1))[np.newaxis,...]
        
        if self.AND_OR == 3:
            # inverter
            filtered_X = m*X
            res = ones - filtered_X

            return res
            
    
class Model():
    
    def __init__(self, n_pop= 5000, cutoff = 5000):
        
        self.layers = []
        self.n_pop = n_pop
        self.cutoff = cutoff
        
    def add_layer(self,dims, n_channels, AND_OR=True, head = False):
        # add output size arg
        
        lyr = gate_layer(dims, n_channels=n_channels, AND_OR=AND_OR)
        self.layers.append(lyr)

        
    def forward(self,x, h, c):
        inv_x = np.ones(x.shape).to(device) - x
        aug_x = np.concatenate([x, inv_x], axis=2)
        
        inv_h = np.ones(h.shape).to(device) - x
        aug_h = np.concatenate([h, inv_h], axis=2)

        
        #print(h_[0,3,:])
        
        xh = np.concatenate([aug_x,aug_h], axis=2)

        
        xh = self.layers[0].forward(xh)
        xh = self.layers[1].forward(xh)
        
        
        i, g, f, o = torch.split(xh,4,axis=2)
       
        
        # g is old c??
        
        #new_c = 1 - (1 - (c * f))*(1-(g* i))
        new_c = c*f + (1 -f)*i 
        new_h = new_c*o
        
        # END LSTM
        
        out = self.layers[2].forward(new_h)
        Y_ = self.layers[3].forward(out)
        
        return new_h, new_c, Y_
    

    
    def params(self):
        # returns a list of the parameters to give to the optimizer
        self.para = []

        for layer in self.layers:
        
            self.para.append(layer.w)
            
        return self.para
    

            
        
    
def recurrency_label(seq_len):
    
    labels = []
    X = []

    X = np.zeros([seq_len,1])
    #X[0,:] = 1.0
    for ii in range(seq_len):
        
        if ii % 4 == 0:
            labels.append(np.ones([1,1]))

        else:
            labels.append(np.zeros([1,1]))

    
    return X, np.concatenate(labels, axis=0)



def evluate_func(model, params):
    model.set_model_params(params)
    loss_cum = 0
    Xs, labels  = recurrency_label(10)
    
#     X = np.ones([1,1])
#     inv_X = 1 - X

#     aug_X = np.concatenate([X, inv_X], axis=1)
    h_ = np.zeros([1,15])
    c_ = np.zeros([1,15])

    h_[0,0] = 1.0
#     xh = np.concatenate([aug_X,h_], axis=1)
#     _, h_ = model.forward(xh)

    for X, label in zip(Xs,labels):
        
        inv_X = 1- X
        
        aug_X = np.concatenate([X, inv_X], axis=0)
        
        xh = np.concatenate([aug_X[np.newaxis,...],h_], axis=1)

        h, c_, Y_= LSTM.forward(x, h, c_)
        
        pred = out[:,0:1]
        h_ = np.copy(out[:,1:])

        #print(label, pred)
        loss = BCE_loss(label, pred)
        loss_cum += loss
    
    return loss_cum




In [6]:
model = Model()
model.add_layer([1,12], n_channels = 20, AND_OR=1)
model.add_layer([1,20], n_channels = 11, AND_OR=2)



NPARAMS = model.num_params()       # make this a 100-dimensinal problem.
NPOPULATION = 4001    # use population size of 101.
MAX_ITERATION = 4000 # run each solver for 5000 generations.


# defines genetic algorithm solver
ga = SimpleGA(NPARAMS,                # number of model parameters
               sigma_init=0.5,        # initial standard deviation
               popsize=NPOPULATION,   # population size
               elite_ratio=0.1,       # percentage of the elites
               forget_best=False,     # forget the historical best elites
               weight_decay=0.00,     # weight decay coefficient
              )

AttributeError: 'Model' object has no attribute 'num_params'

In [ ]:
fit_func = evluate_func
# defines a function to use solver to solve fit_func
def test_solver(solver):
    history = []
    for j in range(MAX_ITERATION):
        solutions = solver.ask()
        fitness_list = np.zeros(solver.popsize)
        #print(solutions)
        for i in range(solver.popsize):
            fitness_list[i] = -fit_func(model,solutions[i])
            #print(fit_func(model,solutions[i]))
            

        solver.tell(fitness_list)
        result = solver.result() # first element is the best solution, second element is the best fitness
        history.append(result[1])
        if (j+1) % 10 == 0:
            print("fitness at iteration", (j+1), result[1])
    print("local optimum discovered by solver:\n", result[0])
    print("fitness score at this local optimum:", result[1])
    return history

ga_history = test_solver(ga)

In [6]:
cmaes = CMAES(NPARAMS,
              popsize=NPOPULATION,
              weight_decay=0.0,
              sigma_init = 0.5
          )
cma_history = test_solver(cmaes)

(2000_w,4000)-aCMA-ES (mu_w=1006.8,w_1=0%) in dimension 460 (seed=103839, Wed Jul 29 20:56:18 2020)


KeyboardInterrupt: 

In [11]:
# defines PEPG (NES) solver
pepg = PEPG(NPARAMS,                         # number of model parameters
            sigma_init=0.5,                  # initial standard deviation
            learning_rate=0.1,               # learning rate for standard deviation
            learning_rate_decay=1.0,       # don't anneal the learning rate
            popsize=NPOPULATION,             # population size
            average_baseline=False,          # set baseline to average of batch
            weight_decay=0.00,            # weight decay coefficient
            rank_fitness=False,           # use rank rather than fitness numbers
            forget_best=False)   
pepg_history = test_solver(pepg)

fitness at iteration 10 -5.060351721810443
fitness at iteration 20 -5.039915157148965
fitness at iteration 30 -5.039915157148965
fitness at iteration 40 -4.786325644061899
fitness at iteration 50 -4.542477245908696
fitness at iteration 60 -4.429039767539853
fitness at iteration 70 -4.251950382815559
fitness at iteration 80 -4.0853189386801185
fitness at iteration 90 -3.9931903311926185
fitness at iteration 100 -3.920931923563131
fitness at iteration 110 -3.8404116734408658
fitness at iteration 120 -3.7180221391675548
fitness at iteration 130 -3.638605300471249
fitness at iteration 140 -3.552462794918037
fitness at iteration 150 -3.4683311834682544
fitness at iteration 160 -3.4211360793875167
fitness at iteration 170 -3.380197272642628
fitness at iteration 180 -3.3586081468875624
fitness at iteration 190 -3.3193830277422918
fitness at iteration 200 -3.2854970680712694
fitness at iteration 210 -3.2587221839715754
fitness at iteration 220 -3.242087785954948
fitness at iteration 230 -3.232

fitness at iteration 1830 -3.1473743125857023
fitness at iteration 1840 -3.1473743125857023
fitness at iteration 1850 -3.1473743125857023
fitness at iteration 1860 -3.1473743125857023
fitness at iteration 1870 -3.1473743125857023
fitness at iteration 1880 -3.1473743125857023
fitness at iteration 1890 -3.1473743125857023
fitness at iteration 1900 -3.1473743125857023
fitness at iteration 1910 -3.1473743125857023
fitness at iteration 1920 -3.1473743125857023
fitness at iteration 1930 -3.1473743125857023
fitness at iteration 1940 -3.1473743125857023
fitness at iteration 1950 -3.1473743125857023
fitness at iteration 1960 -3.1473743125857023
fitness at iteration 1970 -3.1473743125857023
fitness at iteration 1980 -3.1473743125857023
fitness at iteration 1990 -3.1473743125857023
fitness at iteration 2000 -3.1473743125857023
fitness at iteration 2010 -3.1473743125857023
fitness at iteration 2020 -3.1473743125857023
fitness at iteration 2030 -3.1473743125857023
fitness at iteration 2040 -3.14737

fitness at iteration 3620 -3.1473743125857023
fitness at iteration 3630 -3.1473743125857023
fitness at iteration 3640 -3.1473743125857023
fitness at iteration 3650 -3.1473743125857023
fitness at iteration 3660 -3.1473743125857023
fitness at iteration 3670 -3.1473743125857023
fitness at iteration 3680 -3.1473743125857023
fitness at iteration 3690 -3.1473743125857023
fitness at iteration 3700 -3.1473743125857023
fitness at iteration 3710 -3.1473743125857023
fitness at iteration 3720 -3.1473743125857023
fitness at iteration 3730 -3.1473743125857023
fitness at iteration 3740 -3.1473743125857023
fitness at iteration 3750 -3.1473743125857023
fitness at iteration 3760 -3.1473743125857023
fitness at iteration 3770 -3.1473743125857023
fitness at iteration 3780 -3.1473743125857023
fitness at iteration 3790 -3.1473743125857023
fitness at iteration 3800 -3.1473743125857023
fitness at iteration 3810 -3.1473743125857023
fitness at iteration 3820 -3.1473743125857023
fitness at iteration 3830 -3.14737

In [ ]:
solutions = ga.ask()
evluate_func(model, solutions[0])

In [165]:
X = model.forward(np.ones([1,12]))
#model.forward(X[0])
X.shape

(1, 11)

In [49]:
model.lyr_shapes

[array([20, 12]), array([20, 20]), array([11, 20])]

In [50]:
model.num_params()

860